<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_v3_Japanese_English_Translation_sample_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日英・英日翻訳サンプルコード(Japanese-English English-Japanese Translation sample code)


C3TR-Adapter Version3を使って日本語を英語、英語を翻訳するサンプルコードです。  
Sample code to translate Japanese to English and English to Japanese using C3TR-Adapter Version2.

上段メニューの「ランタイム」→「すべてのセルを実行」で実行してください  
Please execute it by clicking "Runtime" -> "Execute All Cells" in the upper menu  


もしくは各セルの[ ]の部分をクリックして実行してください  
Or click on the [ ] part of each cell to execute it.  


In [1]:
%%bash
pip install peft==0.11.1 bitsandbytes==0.43.1 transformers==4.42.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylin

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

model_id = "unsloth/gemma-2-9b-it-bnb-4bit"
peft_model_id = "webbigdata/C3TR-Adapter"

if torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8:
    dtype = torch.bfloat16
else:
    dtype = torch.float16

model = AutoModelForCausalLM.from_pretrained(model_id,  torch_dtype=dtype, device_map="auto")
model = PeftModel.from_pretrained(model = model, model_id = peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [3]:
def trans(my_str):
    input_ids = tokenizer(my_str, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).input_ids.cuda()

    # Translation
    generated_ids = model.generate(input_ids=input_ids,
        num_beams=3, max_new_tokens=800, do_sample=True, temperature=0.5, top_p=0.3,
        use_cache=True,
        )
    full_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return full_outputs[0].split("### Response:\n")[-1].strip()

In [4]:
ret = trans("""You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

<start_of_turn>### Instruction:
Translate Japanese to English.
When translating, please use the following hints:
[writeing_style: web-fiction]

### Input:
恒久平和なんて人類の歴史上なかった。だから私はそんなもの望みはしない。
だが何十年かの平和で豊かな時代は存在できた。我々が次の世代に何か遺産を託さなくてはならないとするなら、やはり平和が一番だ。
そして前の世代から手渡された平和を維持するのは、次の世代の責任だ。
それぞれの世代が、後の世代への責任を忘れないでいれば、結果として長期間の平和が保てるだろう。忘れれば先人の資産は食いつぶされ、人類は一から再出発ということになる。
・・・
要するに私の希望は、たかだかこの先何十年かの平和なんだ。だがそれでも、その十分ノ一の期間の戦乱に勝ること幾万倍だと思う。私の家に１４歳の男の子がいるが、その子が戦場に引き出されるのを見たくない。
ヤン・ウェンリー
<end_of_turn>
<start_of_turn>### Response:
""")
print(ret)

ret = trans("""You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

<start_of_turn>### Instruction:

Translate English to Japanese.
When translating, please use the following hints.
[writing_style: casual]

### Input:
I know I’m Stereotypical Barbie and therefore don’t form conjectures concerning the causality of adjacent unfolding events, but some things have been happening that might be related.
[sighs]
Bad breath this morning, a cold shower, burnt waffle, and falling off my roof.
[gasps] You’re malfunctioning.
What? No, I’m just, I’m…
Am I?
I’ve never seen this kind of malfunction before. It’s usually just hair-related.
You know, you’re gonna have to visit Weird Barbie.
I have never had to visit Weird Barbie.
That’s because you’ve never malfunctioned.
I heard that she used to be the most beautiful Barbie of all, but then someone played with her too hard in the Real World.
[“Spice Up Your Life” playing]
♪ Colors of the world ♪
♪ Spice up your life ♪
♪ Every boy and every girl ♪
♪ Spice up your life ♪
♪ People of the world ♪
♪ Spice up your life ♪
♪ Ahh ♪
♪ Slam me to the left ♪
♪ If you’re having a good time ♪
♪ Shake it to the right ♪
And now she’s fated to an eternity of making other Barbies perfect while falling more and more into disrepair herself.
That, and we all call her Weird Barbie both behind her back and also to her face.
She’s so weird.
Why is she always in the splits?
<end_of_turn>
<start_of_turn>### Response:
""")
print(ret)

Eternal peace has never existed in human history. That's why I don't hope for such a thing.
However, there have been periods of peace and prosperity that lasted for decades. If we are to leave something behind for the next generation, peace is the best option.
And maintaining the peace passed down from the previous generation is the responsibility of the next generation.
If each generation remembers its responsibility to future generations, peace will be maintained over the long term. If they forget, the legacy of our ancestors will be consumed, and humanity will have to start over from scratch.
...
In short, my hope is for peace for the next few decades. But even that is worth a million times more than a period of war. I have a 14-year-old boy in my house, and I don't want to see him dragged into the battlefield.
Yang Wen-li
私はステレオタイプ的なバービーなので、隣接する展開する出来事の因果関係について推測することはありませんが、いくつか関連しているかもしれないことが起きています。
[ため息をつく]
今朝は口臭、冷たいシャワー、焦げたワッフル、そして屋根から転げ落ちました。
[息を呑む] あなたは故障しているわ。
何？いいえ、私はただ、私は…


## 留意事項 Attention
**Do not save this adapter merged with the base model**, as there exists a bug that reduces performance when saving this adapter merged with the model.    

このアダプターをモデルとマージして保存すると性能が下がってしまう不具合が存在するため、**ベースモデルとマージして保存しないでください**

## 謝辞 Acknowledgment

### Referenced models.
Original Base Model  
google/gemma-7b  
https://huggingface.co/google/gemma-7b

Base Model  
unsloth/gemma-7b-bnb-4bit  
https://huggingface.co/unsloth/gemma-7b-bnb-4bit

QLoRA Adapter  
webbigdata/C3TR-Adapter  
https://huggingface.co/webbigdata/C3TR-Adapter


This adapter was trained with Unsloth.  
https://github.com/unslothai/unsloth  

その他、[ALMA](https://arxiv.org/abs/2309.11674)をはじめ、コミュニティの皆さんからヒントを貰っています。ありがとう  
Other tips I have received from [ALMA](https://arxiv.org/abs/2309.11674) and others in the community. Thank you.  

このスクリプトは[webbigdata](https://webbigdata.jp/)によって作成されました  
This script was created by [webbigdata](https://webbigdata.jp/).  